- What is None data type? 
- What is NaN data type?
- How to detect NULL values in Pandas?


In [1]:
# Import libraries and check versions
import pandas as pd
import sys
import missingno as msno
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

print('Python version ' + sys.version)
print('Numpy version ' + np.__version__)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__ )
print('Missingno version ' + msno.__version__)

Python version 3.6.1 |Anaconda custom (x86_64)| (default, Mar 22 2017, 19:25:17) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
Numpy version 1.12.1
Pandas version 0.19.2
Matplotlib version 2.0.0
Missingno version 0.3.5


Create a toy dataset has some missing values.

In [2]:
data = {'first_name': ['Michael', 'Jessica', 'Sue', 'Jake', 'Amy', 'Sally'],
        'last_name': ['Lee', 'Toll', 'Han', np.NaN, None, 'Ken'],
        'age': [14, None, 29 , np.NaN, 52, 45],
        'sex':[None,'F',np.NaN,'F',np.NaN, 'M'],
        }
df = pd.DataFrame(data, columns = ['first_name', 'last_name' ,'sex', 'age'])
df

,first_name,last_name,sex,age
0,Michael,Lee,None,14.0
1,Jessica,Toll,F,NaN
2,Sue,Han,NaN,29.0
3,Jake,NaN,F,NaN
4,Amy,None,NaN,52.0
5,Sally,Ken,M,45.0


an example of usage

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
first_name    6 non-null object
last_name     4 non-null object
sex           3 non-null object
age           4 non-null float64
dtypes: float64(1), object(3)
memory usage: 272.0+ bytes


In [4]:
a = np.array([1,2,np.nan, 4])
a.dtype

dtype('float64')

In [5]:
np.sum(a)

nan

In [6]:
b = np.array([1,2,None, 4])
b.dtype

dtype('O')

In [8]:
#np.sum(b)

How to detect NULL values in Pandas?

- isnull()
- notnull()
- dropna()
- fillna()

In [9]:
df

,first_name,last_name,sex,age
0,Michael,Lee,None,14.0
1,Jessica,Toll,F,NaN
2,Sue,Han,NaN,29.0
3,Jake,NaN,F,NaN
4,Amy,None,NaN,52.0
5,Sally,Ken,M,45.0


In [10]:
df.isnull()

,first_name,last_name,sex,age
0,False,False,True,False
1,False,False,False,True
2,False,False,True,False
3,False,True,False,True
4,False,True,True,False
5,False,False,False,False


In [11]:
# drops the all rows have missing values, note that inplase is False by defult.
df.dropna()

,first_name,last_name,sex,age
5,Sally,Ken,M,45.0


In [14]:
# fills all the missing values with the spcified value, inplase is False.
df['age'].fillna(0)

0    14.0
1     0.0
2    29.0
3     0.0
4    52.0
5    45.0
Name: age, dtype: float64

In [ ]:
df['gender'].fillna(method='')

In [ ]:
msno.matrix(df.sample(5))

In [ ]:
msno.matrix(df.sample(100, replace=True))

In [ ]:
msno.bar(df.sample(5, replace=True))